http://3ms.huawei.com/km/groups/3803117/blogs/details/11856585?l=zh-cn

请设计一个文件下载代理系统，该系统能接收下载请求，并在指定时间内收取多个文件分块后可以进行合并保存。

l  FileSystem(int timeout) —— 初始化系统，有效收包时长为 timeout。
l  Request(int time, int fileId, int number) —— 在时刻 time 收到 fileId 文件的下载请求，分块数量为 number 。
o    若请求队列中已存在编号 fileId 的文件，则不作任何处理，返回 false；
o    若请求队列中不存在编号 fileId 的文件，则将请求加入队尾，返回 true。
·         请求队列用于存放已收到的请求；当文件fileId 完成合并保存或超时出列后，不再重复使用。
l  Receive(int time, int fileId, int partId) —— 在时刻 time，收到 fileId 文件的 partId 分块（编号从 0 ~ number-1）。
o    若请求队列中不存在编号 fileId 的文件、或超时、或收到的分块在同一文件中重复，则返回 -1
o    若请求队列中存在编号 fileId 的文件且收到的分块在该文件中没有重复，则接受该分块，返回该文件尚未收到的分块数量
·         注意：文件收包的有效时间不超过 此文件Request时的time + timeout ，若在有效时间内收完所有包，则文件为 Ready状态，否则视为超时（不再接收分块）。Ready状态文件的合并保存遵循「队首优先保存规则」处理。收包后按照此规则做后处理。

l  Query(int time) —— 查询时刻 time 已完成合并保存的文件个数。注意先按照「队首优先保存规则」处理后再查询。
「队首优先保存规则」如下：

l  任意时刻，对于请求队列队首的文件下载请求，若超时则直接移除该请求；若已收完所有分块，则合并保存文件再移除该请求；持续处理队首的请求，直到队首是未超时且未收完所有分块的请求。
注意：

l  输入保证 Request、Receive、Query 函数的日期 time 参数按输入顺序严格递增

示例 1：

输入：
["FileSystem","request","receive","receive","query"]
[[3],[0,5,2],[1,5,1],[2,5,0],[3]]

输出：[null,true,1,0,1]

解释：
FileSystem sys = FileSystem(3); // 初始化，请求超时的时间间隔为 3
sys.request(0,5,2); // 时刻 0，收到编号 5 的文件下载请求，该文件有 2 个分块，该请求加入队列，返回 true
sys.receive(1,5,1); // 时刻 1，收到编号 5 的文件的分块 1，剩余 1 个未收到的分块，返回 1
sys.receive(2,5,0); // 时刻 2，收到编号 5 的文件的分块 0，剩余 0 个未收到的分块，返回 0
// 时刻 2，由于队首为编号 5 的文件请求，已收完全部分块，因此将这些分块合并为文件 5 保存，并将该请求移出队列
sys.query(3); // 时刻 3，已成功合并保存 1 份完整文件，返回 1
注：输出中的 null 表示此对应函数无输出（其中：C 的构造函数有返回值，但是也是无需输出）

示例 2：

输入：
["FileSystem","request","request","request","receive","receive","receive","request","receive","query","request","request","receive","query","query"]
[[7],[0,17,2],[1,21,2],[2,35,1],[3,21,1],[4,35,0],[5,52,1],[6,21,2],[8,21,0],[9],[10,81,1],[13,76,1],[14,76,0],[16],[17]]

输出：[null,true,true,true,1,0,-1,false,0,2,true,true,0,2,3]

解释：
FileSystem sys = FileSystem(7); // 初始化，请求超时的时间间隔为 7
sys.request(0,17,2); // 返回 true
sys.request(1,21,2); // 返回 true
sys.request(2,35,1); // 返回 true
sys.receive(3,21,1); // 返回 1
sys.receive(4,35,0); // 返回 0，此时编号 35 的文件收完所有包，但不是队首，因此暂时无法合并保存
sys.receive(5,52,1); // 返回 -1
sys.request(6,21,2); // 返回 false
sys.receive(8,21,0); // 时刻 8，编号 21 的文件未超时，收到分块 0，剩余 0 个未收到的分块，返回 0
// 注意，收完包后按照「队首优先保存规则」做后处理。由于队首编号 17 的文件超时，编号 21 的文件成为队首且已收完所有包，合并保存后出列；之后编号 35 的文件成为队首，完成合并保存后也出列
sys.query(9) // 时刻 9，共有编号 21、35 的文件合并保存成功，返回 2
sys.request(10,81,1); // 返回 true
sys.request(13,76,1); // 返回 true
sys.receive(14,76,0); // 返回 0，此时编号 76 的文件收完所有包，但不是队首，因此暂时无法合并保存
sys.query(16); // 返回 2
sys.query(17); // 编号 81 的文件无法在有效时间内收完所有包，视为超时，编号 76 的文件成为队首，完成合并保存后出列。因此返回 3
注：输出中的 null 表示此对应函数无输出（其中：C 的构造函数有返回值，但是也是无需输出）

提示：


1 <= request, receive, query 累计操作数 <= 10^3
2 <= timeout <= 100
0 <= time <= 10^5
1 <= fileId <= 10^3
1 <= number <= 100
0 <= partId < number

 严格递增：对于数组arr，都有 arr[i] < arr[i+1] 。

In [1]:
class FileSystem:

    def __init__(self, timeout: int):
        self.timeout = timeout
        self.file_record = {}
        self.ready = []

    def check(self, time):
        while self.file_record:
            pop_key, (begin_time, number, part_list) = self.file_record.items().__iter__().__next__()
            if number == len(part_list):
                self.file_record.pop(pop_key)
                self.ready.append(pop_key)
            elif begin_time + self.timeout <= time:
                self.file_record.pop(pop_key)
            else:
                break

    def request(self, time, fileId, number):
        if self.file_record.get(fileId):
            return False
        self.file_record[fileId] = [time, number, []]
        return True

    def receive(self, time, fileId, partId):
        if not self.file_record.get(fileId) or self.file_record[fileId][0] + self.timeout < time or partId in \
                self.file_record[fileId][2]:
            return -1
        _, number, part_list = self.file_record[fileId]
        part_list.append(partId)
        self.check(time)
        return number - len(part_list)

    def query(self, time):
        self.check(time)
        return len(self.ready)


FileSystem [7] None

In [2]:
file = FileSystem(7)

request [0, 17, 2] True

In [3]:
file.request(0,17,2)

True

request [1, 21, 2] True

In [4]:
file.request(1,21,2)

True

request [2, 35, 1] True

In [5]:
file.request(2, 35, 1)

True

receive [3, 21, 1] 1

In [6]:
file.receive(3,21,1)

1

receive [4, 35, 0] 0

In [7]:
file.receive(4,35,0)

0

receive [5, 52, 1] -1

In [8]:
file.receive(5,52,1)

-1

request [6, 21, 2] False

In [9]:
file.request(6,21,2)

False

receive [8, 21, 0] 0

In [10]:
file.receive(8,21,0)

0

query [9] 2

In [11]:
file.query(9)

2

request [10, 81, 1] True

In [12]:
file.request(10,81,1)

True

request [13, 76, 1] True

In [13]:
file.request(13,76,1)

True

receive [14, 76, 0] 0

In [14]:
file.receive(14,76,0)

0

query [16] 2

In [15]:
file.query(16)

2

query [17] 3

In [16]:
file.query(17)

3